# Мы, похоже, сделали что-то лишнее?..

Да, описали одновременно и объектную модель (классы), и схему. А ORM позволяют:

* Построить схему БД по объектной модели
  * [Просто в «объектном» (без SQL) стиле](https://docs.sqlalchemy.org/en/14/core/metadata.html)
  * С дополнительными «прибамбасами» — [практически из «нормальных» классов](https://flask-sqlalchemy.palletsprojects.com/en/2.x/quickstart/#a-minimal-application)
* Восстановить объектную модель по схеме БД (в динамических языках — налету), см. ниже.

Какой вариант «правильный»? Зависит от того, что первично — база, или классы с бизнес-логикой

In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

Base = automap_base()
engine = create_engine("sqlite:///students.sqlite3")
session = Session(engine)

А теперь волшебство!

In [2]:
Base.prepare(engine, reflect=True)
students_classes = Base.classes
for c in students_classes:
    print(c)

<class 'sqlalchemy.ext.automap.courses'>
<class 'sqlalchemy.ext.automap.marks'>
<class 'sqlalchemy.ext.automap.students'>
<class 'sqlalchemy.ext.automap.programs'>
<class 'sqlalchemy.ext.automap.programs_courses'>


In [3]:
Program = students_classes.programs
Student = students_classes.students
Course =  students_classes.courses
ProgramsCourses = students_classes.programs_courses
Marks = students_classes.marks

In [5]:
s1 = Student(
    card="x12234",
    surname="Пушкин",
    name="Александр",
    patronymic="Сергеевич"
)
s2 = Student(
    card="xxxxxx",
    surname="Горчаков",
    name="Александр",
    patronymic="Михайлович"
)
l = Program(name="Царскосельский лицей")

l.students_collection += [s1, s2]

with session.begin():
    session.add_all([l, s1, s2])
    # session.commit()  -- само

In [6]:
with session.begin():
    cl = Course(name="Латынь")
    ct = Course(name="Закон Божий")
    pct = ProgramsCourses(programs = l, courses = ct, semester_number = 1)
    pcl = ProgramsCourses(programs = l, courses = cl, semester_number = 1)
    session.add_all([pct, pcl])

In [10]:
for p in session.query(Program):
    print("Программа: ", p.name)
    for pc in p.programs_courses_collection:
        print(" - ", pc.courses.name)

Программа:  математика и информационные технологии
Программа:  теоретическая физика
Программа:  Программная инженерия
Программа:  Царскосельский лицей
 -  Закон Божий
 -  Латынь
